In [18]:
# from scapy.utils import RawPcapReader
# from scapy.layers.l2 import Ether
# from scapy.layers.inet import IP,TCP
import pandas as pd
import numpy as np
import sklearn
import os
import dpkt
from pathlib import Path
from dpkt.ip import IP, IP_PROTO_UDP
from dpkt.udp import UDP
import socket

# import pcapkit
# import pyshark

In [28]:
dataset_path = "C:\hackathon\datasetcr2"
def get_all_dataset_files(path):
    return [os.path.join(dp, f) for dp, dn, filenames in os.walk(path) for f in filenames if os.path.splitext(f)[1] not in ['.txt','.json',".7z",".csv"] ]

In [29]:
all_files = get_all_dataset_files(dataset_path)

In [30]:
rows = []
for file_name in all_files:
    row = {}
    row["File"] = file_name
    if "Ddos_benign" in file_name:
        row["Category"] = "Ddos_benign"
    elif "Ddos_Attack_data" in file_name:
        row["Category"] = "Ddos_Attack_data"
        
    else :
        row["Category"] = "Unknown"
    rows.append(row)

In [31]:
df = pd.DataFrame(rows)

In [32]:
df[df["File"].str.contains("Ddos_Dataset_Part2")]

,File,Category
208,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
209,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
210,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
211,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
212,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
...,...,...
353,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
354,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
355,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown
356,C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,Unknown


In [33]:
for i in df["File"]:
    print(i)

C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\001
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\002
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\003
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\004
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\005
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\006
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\007
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\008
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\009
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\010
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\011
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\012
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\013
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\014
C:\hackathon\datasetcr2\Ddos_Detection_Dataset\D

In [34]:
def get_timeperiod(ts_init,ts):
    return int((ts - ts_init))

In [35]:
# file_name = np.random.choice(df["File"])
stats = {}
for i,file_name in enumerate(df["File"]):
#     if i%10 == 0:
    print(i,file_name)
    f= open(file_name,'rb')
    pcap=dpkt.pcap.Reader(f)
    count_frag=0
    count_public=0
    count_frag_public=0
    irregular_pkt = 0
    pkt_count = 0
    for i,(ts,buf) in enumerate(pcap):
        if i==0:
            ts_init=ts
        eth=dpkt.ethernet.Ethernet(buf)
        #print(dir(eth))
        if eth.type!=dpkt.ethernet.ETH_TYPE_IP:
                continue
        try:
            if eth.type==dpkt.ethernet.ETH_TYPE_IP:
                ip=eth.data

                src_ip = socket.inet_ntoa(ip.src)
                dest_ip = socket.inet_ntoa(ip.dst)
                #print(dir(pkt.data))
                #proto=pkt.transport_layer  

                #print(dir(src))
                more_fragments = bool(ip.off & dpkt.ip.IP_MF)
                fragment_offset = ip.off & dpkt.ip.IP_OFFMASK
                isFragmented = ip.off == more_fragments 

                if ip.p==dpkt.ip.IP_PROTO_TCP and hasattr(ip.data,"sport") : # Check for TCP packets
                    src_pt = ip.data.sport
                    bytes_string= ip.data.data
                    #print("byte:",byte_string)
                    ascii_string= bytes_string.decode("ascii",errors="backslashreplace")
                    #print("TCP",ascii_string)
                elif ip.p==dpkt.ip.IP_PROTO_UDP and  hasattr(ip.data,"sport"):
                    #print(ip.offset)# Check for UDP packets
                    src_pt = ip.data.sport
                    bytes_string= ip.data.data
                    ascii_string= bytes_string.decode("ascii",errors="backslashreplace")
                elif ip.p==dpkt.ip.IP_PROTO_UDP:
                    src_pt = "-100"
                    if hasattr(ip,"data"):
                        bytes_string = ip.data
                        ascii_string = bytes_string.decode("ascii",errors="backslashreplace")
                    else:
                        bytes_string = ""
                        ascii_string=""

                public_flag = ("public" in ascii_string) and ("MIB" in ascii_string)
                private_flag = "private" in ascii_string
                ts = get_timeperiod(ts_init, ts)
                stat = stats.get((src_ip,dest_ip,ts),{"File":file_name,"public_check":False,"payloads":set()})
                if public_flag:
                    stat["public_check"] = True
                stat["cnt"] = stat.get("cnt",0) + 1
                stat["payloads"].add(hash(ascii_string))
                stats[(src_ip,dest_ip,ts)] = stat
                pkt_count +=1
      ###print(i+1)
        except Exception as e:
    #         if ip.p == dpkt.ip.IP_PROTO_TCP:
            print(e)
            irregular_pkt += 1
            pass
    f.close()
#     filename=file_name+".csv"
#     output_file= open(filename,"w")
    
#     for i,stat in stats.items():
#         csv_content= i[0],i[1],i[2],stat["public_check"],stat["cnt"],len(stat["payloads"])
#         output_file.write(str(csv_content))
#         output_file.write('\n')
#     output_file.close()

0 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\001
1 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\002
2 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\003
3 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\004
4 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\005
5 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\006
6 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\007
7 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\008
8 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\009
9 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\010
10 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\011
11 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\012
12 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\013
13 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_Attack_data\014
14 C:\hackathon\

114 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\11.pcap
115 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\12.pcap
116 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\13.pcap
117 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\14.pcap
118 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\15.pcap
119 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\16.pcap
120 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\17.pcap
121 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\18.pcap
122 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\2.pcap
123 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\3.pcap
124 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\4.pcap
125 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_benign\benign3\5.pcap
126 C:\hackathon\datasetcr2\Ddos_Detection_Dataset\Ddos_

220 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\102
221 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\103
222 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\104
223 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\105
224 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\107
225 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\108
226 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\109
227 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\11
228 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\110
229 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\111
230 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\112
231 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\113
232 C:\hackathon\datasetcr2\Ddos_Detectio

324 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\65
325 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\66
326 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\67
327 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\68
328 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\69
329 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\70
330 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\71
331 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\72
332 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\73
333 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\74
334 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\75
335 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_Part2\Ddos_Dataset_Part2\76
336 C:\hackathon\datasetcr2\Ddos_Detection_Dataset_P

{('172.16.0.5',
  '192.168.50.1',
  0): {'File': 'C:\\hackathon\\datasetcr2\\Ddos_Detection_Dataset_Part2\\Ddos_Dataset_Part2\\99', 'public_check': False, 'payloads': {0,
   2149928781641449473,
   6679116504492703744,
   6173652641590886399,
   8577913582529347585,
   3813824564715823104,
   7152796988190973950,
   1977810345189523460,
   142320609247780868,
   -7091928140876611571,
   8721586033867792391,
   2672778078382424082,
   7149507189024710673,
   8600013159732420629,
   -6204636663066075109,
   -5027979967514443741,
   -5204715788671262684,
   304064832828547105,
   3205499532787744803,
   -3716369650336202714,
   1532549662859583527,
   -3493271920357294037,
   -6090384429910876114,
   2026866699125252141,
   -4032789935235882961,
   9155794929726717996,
   -1358340779037327315,
   9040708496395026478,
   -3084621767291551697,
   6546073676040347699,
   8011094490202791987,
   7518961271149486132,
   1850167656192442426,
   5093937817861988409,
   -4603053801459040195,
   9

In [36]:
rows = []
for i in stats:
    row = {"Connection":i}
    row.update(stats[i])
    rows.append(row)
df = pd.DataFrame(rows)

In [37]:
# df.to_csv("cache.csv",index=False)

In [43]:
df.columns

Index(['Connection', 'File', 'public_check', 'payloads', 'cnt', 'Class'], dtype='object')

In [168]:
df = pd.read_csv("cache.csv")
df["Class"] =   df["File"].str.contains("benign") 
df["Class"] = df["Class"].map({False:"DDos",True:"Benign"})
df["Payload_cnt"] = df["payloads"].apply(lambda x:len(x))
df["Unique_ratio"] = df["cnt"]/df["Payload_cnt"]
df.drop(columns="payloads",inplace=True)

In [169]:
# transformed_df = df.groupby(["File","Connection"])[[ 'public_check', 'Payload_cnt', 'cnt']].max()

In [229]:
from sklearn.ensemble import IsolationForest
from ast import literal_eval as make_tuple
def train_classifer(df):
    df["Connection"] = df["Connection"].apply(lambda x : make_tuple(x)[:2])
    print(df["Connection"])
    grouped = df.groupby('Connection')
    clf = IsolationForest(random_state=1234,max_samples=500)
#     for name,group in grouped:
#         print(name)
    x = [group.median() for name,group in grouped]
    print(x)
    clf.fit(x)
    return clf


In [230]:
np.random.seed(0)
train_files = list(np.random.choice(df[df["Class"] == "DDos"]["File"],10)) #+ list( np.random.choice(df[df["Class"] == "Benign"]["File"],1))
# train_files = np.random.choice(df["File"],100)
print(len(train_files))
train_df = df[df["File"].isin(train_files)]
# train_df = pd.concat([train_df[train_df["Class"] == "DDos"].sample(n=100),train_df[train_df["Class"] == "Benign"].sample(n=1000)])
print(len(train_df))
clf = train_classifer(train_df)

10
44195


C:\Users\Akash\anaconda3\envs\pcap\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


1918          (192.168.50.7, 72.21.91.29)
1919          (72.21.91.29, 192.168.50.7)
1920          (192.168.50.7, 52.36.71.24)
1921       (192.168.50.7, 125.56.201.115)
1922          (52.36.71.24, 192.168.50.7)
                        ...              
7733671    (192.168.50.8, 172.217.12.142)
7733672    (172.217.12.142, 192.168.50.8)
7733673    (52.200.108.113, 192.168.50.8)
7733674    (172.217.12.142, 192.168.50.8)
7733675    (192.168.50.8, 172.217.12.142)
Name: Connection, Length: 44195, dtype: object
[public_check     0.000000
cnt              1.000000
Payload_cnt     22.000000
Unique_ratio     0.047619
dtype: float64, public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64, public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64, public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64, public_check    0.0000

In [226]:
import joblib
joblib.dump(clf,"pcap_model.joblib")

['pcap_model.joblib']

In [227]:
train_df["Class"].value_counts()

DDos    44195
Name: Class, dtype: int64

In [233]:
for name,group in train_df.groupby("Connection"):
    
    print(group.median(),clf.predict([group.median()]))

public_check     0.000000
cnt              1.000000
Payload_cnt     22.000000
Unique_ratio     0.047619
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             4.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.000000
cnt              6.000000
Payload_cnt     13.500000
Unique_ratio     0.541667
dtype: float64 [1]
public_check    0.000000
cnt             4.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     22.500000
Unique_ratio     0.156061
dtype: float64 [1]
public_check    0.000000
cnt        

public_check     0.00
cnt              4.00
Payload_cnt     22.00
Unique_ratio     0.16
dtype: float64 [1]
public_check      0.00000
cnt              13.00000
Payload_cnt     110.00000
Unique_ratio      0.11039
dtype: float64 [1]
public_check    0.000000
cnt             4.000000
Payload_cnt     3.000000
Unique_ratio    1.333333
dtype: float64 [-1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             4.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     23.000000
Unique_ratio     0.178571
dtype: float64 [1]
public_check     0.00
cnt              4.00
Payload_cnt     22.00
Unique_ratio     0.16
dtype: float64 [1]
public_check     0.000000
cnt              3.000000
Payload_cnt     12.000000
Unique_ratio     0.424242
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload

public_check     0.00
cnt              4.00
Payload_cnt     22.00
Unique_ratio     0.16
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     42.000000
Unique_ratio     0.105651
dtype: float64 [1]
public_check     0.000000
cnt              6.000000
Payload_cnt     25.000000
Unique_ratio     0.193723
dtype: float64 [1]
public_check     0.000000
cnt              3.000000
Payload_cnt     21.000000
Unique_ratio     0.116225
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     21.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     22.000000
Unique_ratio     0.163333
dtype: float64 [1]
public_check    0.000000
cnt             4.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.0
cnt             6.0
Payload_cnt     3.0
Unique_ratio    2.0
dtype: float64 [-1]
public_check    0.000000
cnt             2.000000
Payload_c

public_check     0.000000
cnt              4.000000
Payload_cnt     21.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     24.500000
Unique_ratio     0.137413
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     24.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     23.000000
Unique_ratio     0.135921
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     23.000000
Unique_ratio     0.135195
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     23.000000
Unique_ratio     0.145217
dtype: float64 [1]
public_check     0.00
cnt              4.00
Payload_cnt     21.50
Unique_ratio     0.16
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     22.000000
Unique_ratio     0.118242
dtype: float64 [1]
public_check     0.000000
cnt   

public_check     0.0
cnt             10.0
Payload_cnt     25.0
Unique_ratio     0.4
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.00000
cnt              8.00000
Payload_cnt     12.00000
Unique_ratio     1.52381
dtype: float64 [-1]
public_check    0.000000
cnt             4.000000
Payload_cnt     3.000000
Unique_ratio    1.333333
dtype: float64 [-1]
public_check     0.000000
cnt              1.000000
Payload_cnt     22.000000
Unique_ratio     0.045455
dtype: float64 [1]
public_check    0.0
cnt             3.0
Payload_cnt     3.0
Unique_ratio    1.0
dtype: float64 [-1]
public_check     0.000000
cnt              2.000000
Payload_cnt     21.000000
Unique_ratio     0.095238
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     13.500000
Unique_ratio     0.779026
dtype: float64 [1]
public_check     0.000000
cnt              2.000000
Payload_cnt     2

...

dtype: float64 [1]
public_check     0.000000
cnt              8.000000
Payload_cnt     21.000000
Unique_ratio     0.626866
dtype: float64 [1]
public_check     0.00000
cnt              6.00000
Payload_cnt     65.00000
Unique_ratio     0.09375
dtype: float64 [1]
public_check     0.0
cnt             18.0
Payload_cnt     90.0
Unique_ratio     0.2
dtype: float64 [1]
public_check     0.000000
cnt              6.000000
Payload_cnt     24.500000
Unique_ratio     0.177778
dtype: float64 [1]
public_check     0.000000
cnt              2.000000
Payload_cnt     21.000000
Unique_ratio     0.095238
dtype: float64 [1]
public_check     0.000000
cnt              2.000000
Payload_cnt     21.000000
Unique_ratio     0.095238
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     43.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check     0.000000
cnt              6.000000
Payload_cnt     34.500000
Unique_ratio     0.172222
dtype: float64 [1]
public_check     0.00

public_check     0.000000
cnt              4.000000
Payload_cnt     21.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check     0.000000
cnt              5.500000
Payload_cnt     46.000000
Unique_ratio     0.118375
dtype: float64 [1]
public_check     0.00
cnt              4.00
Payload_cnt     22.00
Unique_ratio     0.16
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     22.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     22.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     23.000000
Unique_ratio     0.146247
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     42.000000
Unique_ratio     0.133333
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     22.500000
Unique_ratio     0.190476
dtype: float64 [1]
public_check     0.000000
cnt   

public_check     0.000000
cnt              4.000000
Payload_cnt     24.500000
Unique_ratio     0.163333
dtype: float64 [1]
public_check     0.00
cnt              4.00
Payload_cnt     25.00
Unique_ratio     0.16
dtype: float64 [1]
public_check     0.00
cnt              4.00
Payload_cnt     24.00
Unique_ratio     0.16
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     24.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt 

public_check     0.000000
cnt              4.000000
Payload_cnt     23.000000
Unique_ratio     0.173913
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     24.000000
Unique_ratio     0.166667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt            

public_check     0.00000
cnt              7.00000
Payload_cnt     45.50000
Unique_ratio     0.17029
dtype: float64 [1]
public_check     0.000000
cnt              4.000000
Payload_cnt     24.000000
Unique_ratio     0.173913
dtype: float64 [1]
public_check    0.000000
cnt             8.000000
Payload_cnt     3.000000
Unique_ratio    1.333333
dtype: float64 [-1]
public_check    0.000000
cnt             8.000000
Payload_cnt     3.000000
Unique_ratio    1.333333
dtype: float64 [-1]
public_check    0.000000
cnt             6.000000
Payload_cnt     3.000000
Unique_ratio    1.333333
dtype: float64 [-1]
public_check    0.000000
cnt             2.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check    0.000000
cnt             4.000000
Payload_cnt     3.000000
Unique_ratio    0.666667
dtype: float64 [1]
public_check     0.000000
cnt              7.000000
Payload_cnt     46.000000
Unique_ratio     0.722846
dtype: float64 [1]
public_check    0.000000
cnt         

In [231]:
train_df

,Connection,File,public_check,cnt,Class,Payload_cnt,Unique_ratio
1918,"(192.168.50.7, 72.21.91.29)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,40,DDos,67,0.597015
1919,"(72.21.91.29, 192.168.50.7)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,34,DDos,46,0.739130
1920,"(192.168.50.7, 52.36.71.24)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,2,DDos,21,0.095238
1921,"(192.168.50.7, 125.56.201.115)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,2,DDos,21,0.095238
1922,"(52.36.71.24, 192.168.50.7)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,2,DDos,3,0.666667
...,...,...,...,...,...,...,...
7733671,"(192.168.50.8, 172.217.12.142)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,8,DDos,25,0.320000
7733672,"(172.217.12.142, 192.168.50.8)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,6,DDos,24,0.250000
7733673,"(52.200.108.113, 192.168.50.8)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,15,DDos,113,0.132743
7733674,"(172.217.12.142, 192.168.50.8)",C:\hackathon\datasetcr2\Ddos_Detection_Dataset...,False,2,DDos,21,0.095238
